# 1. Set-up

In [ ]:
# a temporary solution, run twice to set the current path as '../Fair-RL/src'
%cd ..

In [3]:
%cd src

/Users/xiaohui/Documents/Code/Fair-RL/src


In [4]:
# used to auto-reload imported module after making modification inside .py files
%load_ext autoreload
%autoreload 2

## 1.1 imports

In [5]:
from experiments.configs.base import params
from env.mrp_env_rccc import MachineReplacement
from solver.ggf_dlp import build_dlp, extract_dlp, solve_dlp
from utils.common import MDP4LP
from utils.mrp import MRPData
from utils.policy import calculate_state_value
import pandas as pd
import numpy as np
from utils.encoding import state_vector_to_int_index, state_int_index_to_vector
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
params.weight_type = 'exponential3'

In [40]:
# set parameters
params.num_actions = 2
params.num_states = 3
params.num_groups = 2

mrp_data = MRPData(
    num_groups=params.num_groups,
    num_states=params.num_states,
    num_actions=params.num_actions,
    prob_remain=params.prob_remain,
    weight_type=params.weight_type,
    cost_types_operation=params.cost_type_operation,
    cost_types_replace=params.cost_type_replace,
)

mdp = MDP4LP(
    num_states=mrp_data.num_global_states,
    num_actions=mrp_data.num_global_actions,
    num_groups=mrp_data.num_groups,
    transition=mrp_data.global_transitions,
    costs=mrp_data.global_costs,
    discount=params.gamma,
    weights=mrp_data.weights,
    minimize=True,
    encoding_int=False,
    base_num_states=params.num_states,
)

In [41]:
# calculate LP values
uniform_dist = [1 / mrp_data.num_global_states for i in range(mrp_data.num_global_states)]
model = build_dlp(mdp=mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model, _ = solve_dlp(model=model)
results = extract_dlp(model=model, print_results=True)

Initial distribution:  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]
Solver solving time: 0.0085
Number of solutions found: 1
Proportion of stochastic policy: 20.0%
Policy:                        Var X:            Costs - Group 1 | Group 2                        
        0    1    2          0       1       2           0    1     2        0     1    2
(0, 0)  1    0    0     3.2918       0       0        0.00  1.0  0.00     0.00  0.00  1.0
(0, 1)  1    0    0     3.9055       0       0        0.00  1.0  0.00     0.17  0.17  1.0
(0, 2)  0    0    1          0       0  0.9698        0.00  1.0  0.00     0.67  0.67  1.0
(1, 0)  1    0    0     3.9055       0       0        0.17  1.0  0.17     0.00  0.00  1.0
(1, 1)  1    0    0     4.6161       0       0        0.17  1.0  0.17     0.17  0.17  1.0
(1, 2)  0    0    1          0       0  1.0274        0.17  1

In [8]:
from utils.count import CountMDP

In [9]:
count_mrp = MRPData(
    num_groups=params.num_groups,
    num_states=params.num_states,
    num_actions=params.num_actions
)

In [ ]:
count_mrp